# Mastro Feature-store client example

Firstly import the module:

In [106]:
from mastro_fs.client import MastroFeatureStoreClient
from mastro_fs.dto import FeatureSet, Feature

Then establish a connection with a feature store instance (e.g., localhost:8086). 

In [107]:
c = MastroFeatureStoreClient('127.0.0.1', 8086)

We suggest using the available docker compose file to quickly ramp up a test service. Alternatively, there exists a Helm chart shall you wish deploying Mastro to a K8s cluster.

## Creating Features

In [108]:
f1 = Feature(name="feature1", value=10, data_type="int")
f2 = Feature(name="feature2", value=True, data_type="bool")

In [109]:
fs = FeatureSet(
    name="myfirstfeatureset", 
    version="test-v1.0", 
    description="", 
    labels = {},
    features = [f1, f2]
)

In [110]:
c.create_featureset(fs)

## Retrieving Featuresets

In [111]:
fs = c.get_featureset_by_name("myfirstfeatureset")

In [112]:
fs.data

In [113]:
for f in fs.data:
    print(f.name, f.version)

myfirstfeatureset test-v1.0


List all is a debug-only method that can be used to return all feature sets in the store:

In [114]:
c.list_all(limit=10, page=1)

## Serializing Pandas Dataframes

In [115]:
import pandas as pd
df = pd.DataFrame(
    [["a", "b"], ["c", "d"]],
    index=["row 1", "row 2"],
    columns=["col 1", "col 2"],
)

In [116]:
df.to_dict('split')

{'index': ['row 1', 'row 2'],
 'columns': ['col 1', 'col 2'],
 'data': [['a', 'b'], ['c', 'd']]}

In [117]:
f3 = Feature(name="feature3", value=df.to_dict('split'), data_type="pandas_dataframe")

In [118]:
fs = FeatureSet(
    name="myfirstfeatureset", 
    version="test-v1.0", 
    description="", 
    labels = {
        "refers-to" : "project-gilberto",
        "environment" : "test"
    },
    features = [f3]
)

In [119]:
c.create_featureset(fs)

In [120]:
assert c.get_featureset_by_name("myfirstfeatureset") is not None

In [121]:
c.get_featureset_by_name("myfirstfeatureset")

In [122]:
for f in c.get_featureset_by_name("myfirstfeatureset").data:
    print(f.name, f.version)
    for feature in f.features:
        print(feature.name, feature.value, type(feature.value))

myfirstfeatureset test-v1.0
feature1 10 <class 'int'>
feature2 True <class 'bool'>
myfirstfeatureset test-v1.0
feature3 {'columns': ['col 1', 'col 2'], 'data': [['a', 'b'], ['c', 'd']], 'index': ['row 1', 'row 2']} <class 'dict'>
